In [16]:
import torch
import torch.nn as nn
import torch.optim as optim    #optim.lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset
import torchvision #torchvision.datasets, torchvision.models, torchvision.transforms
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau #Reduce learning rate when a metric has stopped improving

import numpy as np


import matplotlib.pyplot as plt
import datetime, os, copy
from pathlib import Path
from collections import OrderedDict, namedtuple

# 'PIL' is the Python Imaging Library, 
# 'Image' module provides a class to represent a PIL image. 
## it provides factory functions, like load images from files, and to create new images.
from PIL import Image


#Used to compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
from sklearn.metrics.ranking import roc_auc_score

from sklearn.preprocessing import normalize


In [17]:
print(torch.__version__)

0.3.0


In [18]:
def load_raw(path, name):
    file = Path(path) / name
    if file.exists():
        return np.load(file,encoding='bytes')
    else:
        raise Exception("File not found chutia!!")

In [19]:
def to_tensor(numpy_array):
    # Numpy array -> Tensor
    return torch.from_numpy(numpy_array)


def to_variable(tensor):
    # Tensor -> Variable (on GPU if possible)
    if torch.cuda.is_available():
        # Tensor -> GPU Tensor
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)

In [20]:
#taken from github of arnoweng/CheXNet
def compute_AUCs(y_hat, y_true, args):
    AUROCs = []
    
    y_hat = normalize(y_hat, axis=1, norm='l1')
    print (y_true.shape)
    print (y_hat.shape)
    #print (roc_auc_score(y_true, y_hat))
    for i in range(args.n_classes):
        AUROCs.append(roc_auc_score(y_true[:, i], y_hat[:, i]))
        
    AUROC_avg = np.array(AUROCs).mean()
    print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
    for i in range(args.n_classes):
        print('The AUROC of {} is {}'.format(args.disease_categories[i], AUROCs[i]))

## MODEL

In [21]:
class DenseNet121(nn.Module):
    #!!!def __init__(self, classCount, isTrained):
    def __init__(self, args):
        #!!!super(DenseNet121, self).__init__()
        super().__init__()
        self.densenet121 = None
        
        if args.backprop_pretained:
            #Fixed Feature Extractor
            #freeze the weights for all of the network except that of the final fully connected layer. 
            
            self.densenet121 = torchvision.models.densenet121(pretrained=True)
            for param in self.densenet121.parameters():
                param.requires_grad = False
            
            #parameters = filter(lambda p: p.requires_grad, self.densenet121.parameters())
            #for param in parameters:
            #    param.requires_grad = False
                
        else:
            #Finetuning 
            #initialize the network with a pretrained networt. Rest of the training looks as usual.
            self.densenet121 = torchvision.models.densenet121(pretrained=True)
            
        
        # Parameters of newly constructed modules have requires_grad=True by default
        #fc -> contains the last layer of network (only for resnet)
        #classifier -> -> contains the last layer of network (only for densenet)
        
        ##in RESNET last layer is from 2048 to 1000
        #num_features = dcnn.fc.in_features 
        
        ##in DENSENET last layer is from 1024 to 1000
        num_features = self.densenet121.classifier.in_features
        
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_features, args.n_classes),
            nn.Sigmoid()
        )
        
        
    #each image should be size 224x224 as original paper of Densenet states
    def forward(self, input_val):
        y = self.densenet121(input_val)
        return y
    
                    
    def initialize_weigths(self, args):
        pass


## Dataset

In [22]:
#class ChestXray(torch.utils.data.Dataset):
class ChestXray (torch.utils.data.TensorDataset):
    def __init__(self, args, dataset_list_file, path, is_val=False):
        
        
        #normalize = transforms.Normalize([12.69, 12.69, 12.69],
        #                                 [5.85, 5.85, 5.85])
        
        normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                         [0.229, 0.224, 0.225])
    
        #used in variation of CheXnet of Weng,Zhuang,Tian
        self.transform1 = transforms.Compose([
                                transforms.Resize(256),
                                transforms.TenCrop(224), #return  list of 10 images 
                                transforms.Lambda (lambda crops: torch.stack([transforms.ToTensor()(x) for x in crops])),
                                transforms.Lambda(lambda crops: torch.stack([normalize(x) for x in crops]))
                            ])
        
        #used in original paper of CheXnet of Rajpurkar,Irvin,Zhu,Ng
        self.transform2 = transforms.Compose([
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                normalize
                            ])
        self.is_val = is_val
            
        image_names = []
        labels = []
        tmp = "all_images"
        #tmp = "../all_images/images"
        with open(os.path.join(path, dataset_list_file), "r") as file:
            for line in file:
                items = line.split(',')
                label = [int(i) for i in items[1:]]
                labels.append(label)

                image_filename = items[0]
                image_name = os.path.join(path,tmp, image_filename)
                #image_name = os.path.join(tmp, image_filename)
                image_names.append(image_name)
                
        self.image_names = image_names
        self.labels = labels
        print ("There are {} images in the Images Dataset".format(len(self.image_names)))
        
    def __getitem__(self, index):

        
        image_filename = self.image_names[index]
        image = Image.open(image_filename).convert('RGB')
        label = torch.FloatTensor(self.labels[index])
        if args.transform == "transform_1":
            if self.is_val == False:
                image = self.transform1(image)
            else:
                image = self.transform2(image)
        elif args.transform == "transform_2":
            image = self.transform2(image)
        return image, label

    def __len__(self):
        return len(self.image_names)
    


## Validate Routine

In [23]:
#routine to compute loss based on Validation dataset
def validate_routine(model, args, val_datalist,path):
    
    model.eval() #DO NOT FORGET to do evaluation
    
    loss = nn.BCELoss()
    
    dataset = ChestXray(args, val_datalist, path, is_val=True)
    data_loader = torch.utils.data.DataLoader(
                    dataset, batch_size=args.batch_size, shuffle=False,
                    num_workers=args.num_workers, pin_memory=args.pin_memory)
    
    losses = []
    for i,(input_val,labels) in enumerate(data_loader): 
            
        #forward pass
        #print ("val batch processing: ",i)
        prediction = model(to_variable(input_val))

        #print("Finished forward pass")
        val_loss = loss(prediction, to_variable(labels))
        losses.append(val_loss.data.cpu().numpy())
            
    return losses

## Train Routine

In [24]:
def training_routine(args, path, train_datalist, val_datalist):
    
    #open files and load content
    
    # Create the network
    if args.load_disk_model==True:
        
        
        load_model = torch.load(args.load_checkpoint_filename, map_location=lambda storage, loc: storage)
        #load_model = torch.load('ChexNet_model2.pytorch')
        model = DenseNet121(args)
        new_state_dict = OrderedDict()
        
        tmp = list(model.state_dict().keys())
            
        for i,(k, v)  in enumerate(load_model.items()):
            #print (k)
            name = k[7:] # remove `module.`
            new_state_dict[tmp[i]] = v
        
        # load params
        model.load_state_dict(new_state_dict)
    elif args.load_disk_model == False:
        model = DenseNet121(args)  
    
    #Initialize weitgths
    #my_model.initialize_weigths()
    
    
    #Choose the loss function / optimizer
    loss = nn.BCELoss(size_average = True)
    
    #choose optimizer
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                            lr=args.learn_rate,
                            weight_decay=args.l2)
#     optim = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
#                              lr=args.learn_rate,
#                              weight_decay=args.l2,
#                              momentum = 0)
    scheduler = ReduceLROnPlateau(optim, factor = 0.1, patience = 5, mode = 'min')
                         
    print ("Created Neural Network arquitecture")
    
    if torch.cuda.is_available():
        # Move the network and the optimizer to the GPU
        print ("Moving to GPU")
        model = model.cuda()
        model = torch.nn.DataParallel(model).cuda()
        loss = loss.cuda()
    
    dataset = ChestXray(args, train_datalist, path, is_val=False)
    
    
    data_loader = torch.utils.data.DataLoader(
                    dataset, batch_size=args.batch_size, shuffle=True,
                    num_workers=args.num_workers, pin_memory=args.pin_memory)
    
    print ("Created data objects")
    val_losses = []
    losses= []
    for epoch in range(args.epochs): 
        model.train()
        t0 = datetime.datetime.now()
        losses = []
        for i,(input_val,labels) in enumerate(data_loader): 

            #if transform_1 using, we got a vector of 5 DIM
            #and we only need a 4 DIM
            if args.transform == 'transform_1':  
                bs, n_crops, c, h, w = input_val.size() #e.g(4,10,3,224,244)
                input_val = input_val.view(-1, c, h, w) #e.g(40,3,224,224)
                bs, n_classes = labels.size() #e.g(4,14)
                labels = labels.unsqueeze(2).repeat(1, n_crops, 1 )
                labels = labels.contiguous().view(bs*n_crops, n_classes)
                
            prediction = model(to_variable(input_val))

            #print("Finished forward pass")
            #print (prediction.shape)
            
            train_loss = loss(prediction, to_variable(labels))
            optim.zero_grad()# Reset the gradients NEVER FORGET THIS
            train_loss.backward()
            optim.step() # Update the network
            
            losses.append(train_loss.data.cpu().numpy())

            
            if i % 400 == 0:
                print('Minibatch ',i,train_loss.data.cpu().numpy())
            #if i % 1200 == 0:
                #val_loss = validate_routine(model, args, val_datalist, path)
                #scheduler.step(np.asscalar(np.mean(val_loss)))
                #model.train()
            
        val_loss = np.asscalar(np.mean(validate_routine(model, args, val_datalist, path)))   
        
        val_losses.append(val_loss)
        print ('EPOCH', end='\t')
        print ("Epoch {} Train Loss: {:.4f}".format(epoch, np.asscalar(np.mean(losses))), end='\t')
        print ("Epoch {} Validation Loss: {:.4f}".format(epoch, val_loss), end='\t')
        print ("Epoch Time: {}".format(datetime.datetime.now()-t0))
        if min(val_losses) == val_loss:
            print ("Saving model on Disk")
            torch.save(model.state_dict(), args.save_checkpoint_filename)
        torch.save(model.state_dict(), args.save_checkpoint_filename2)
    return model,losses


## Parameters

In [25]:
disease_categories = {'Atelectasis': 0, 'Cardiomegaly': 1, 'Effusion': 2,
                          'Infiltration': 3, 'Mass': 4, 'Nodule': 5, 'Pneumonia': 6,
                          'Pneumothorax': 7, 'Consolidation': 8, 'Edema': 9,
                          'Emphysema': 10, 'Fibrosis': 11, 'Pleural_Thickening': 12, 'Hernia': 13,
                          }

disease_categories2 = {val:key for (key, val) in disease_categories.items()}

col_nanes = ['FileName', 'Atelectasis', 'Cardiomegaly', 'Effusion', 
             'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 
             'Pneumothorax', 'Consolidation', 'Edema', 
             'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'
            ]


In [26]:
#in GPU
args = {'load_disk_model':True, 'load_checkpoint_filename':'ChexNet_model2.pytorch',
        'save_checkpoint_filename':'ChexNet_model4.pytorch',
        'save_checkpoint_filename2':'ChexNet_model5.pytorch',
        'backprop_pretained':False, #if False do finetuning if True just Fixed Feature Extractor
        'n_classes':14, 'transform':'transform_2', 'disease_categories':disease_categories2,
        'num_workers':16, 'pin_memory':True, #used for DataLoader
        'batch_size':32, 'epochs':2,
        'learn_rate':0.00001, 'l2':0.1e-08, #used in optimization 
       }

#in CPU
#args['num_workers'] = 16
#args['pin_memory'] = True

args = namedtuple('args', args.keys())(**args)

path = "dataset"
train = "train_set.csv" 
val = "val_set.csv"
test = "test_set.csv"

In [ ]:
a = datetime.datetime.now()
print (a)
print ("Start Training")
model,losses = training_routine(args, path, train, val)
print ("I am done training")
b = datetime.datetime.now()
print (b)
print (b-a)


### PREDICT TEST

In [ ]:
use_from_disk = True
if use_from_disk:
    load_model = torch.load(args.load_checkpoint_filename, map_location=lambda storage, loc: storage)
    #load_model = torch.load('ChexNet_model2.pytorch')
    model = DenseNet121(args)
    new_state_dict = OrderedDict()
    for k, v in load_model.items():
        #print (k)
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    model.load_state_dict(new_state_dict)
    #model.load_state_dict(load_model)
    if torch.cuda.is_available():
        # Move the network and the optimizer to the GPU
        print ("Moving to GPU")
        model = model.cuda()
        model = torch.nn.DataParallel(model).cuda()
    #print(model) 


t0 = datetime.datetime.now()

model.eval() #DO NOT FORGET to do evaluation


dataset = ChestXray(args, test, path, is_val=True)
data_loader = torch.utils.data.DataLoader(
                dataset, batch_size=args.batch_size, shuffle=False,
                num_workers=args.num_workers, pin_memory=args.pin_memory)


y_hat = []
y_true = []
for i,(input_val,labels) in enumerate(data_loader): 

    #forward pass
    if i % 50 == 0:
        print ("val batch processing: ",i)
                
    prediction = model(to_variable(input_val))
    #print ("Donde Forward Pass")
    y_true.append(labels.cpu().numpy().astype(int))
    y_hat.append(prediction.data.cpu().numpy())
    

y_hat = np.vstack(y_hat)
y_true = np.vstack(y_true)
compute_AUCs(y_hat, y_true, args)
print ("Predict Time: {}".format(datetime.datetime.now()-t0))

### HEATMAP GENERATOR

In [ ]:

image = Image.open("dataset/all_images/00000001_002.png").convert('RGB')
args['batch_size'] = 1

weights = list(model.parameters())[-2]
print (weights.shape)



heatmap = None
for i in range (weights.shape[0]):
    map = output[0,i,:,:]
    if i == 0: heatmap = self.weights[i] * map
    else: heatmap += self.weights[i] * map

In [82]:
weights.shape

torch.Size([14, 1024])